In [1]:
import sys
sys.path.append('../..')
import os
sys.path.append(os.getcwd())
import numpy as np
from minst.core import *
print(sys.path)

['f:\\py\\nemat\\minst\\test', 'c:\\Users\\Administrator\\anaconda3\\python312.zip', 'c:\\Users\\Administrator\\anaconda3\\DLLs', 'c:\\Users\\Administrator\\anaconda3\\Lib', 'c:\\Users\\Administrator\\anaconda3', '', 'c:\\Users\\Administrator\\anaconda3\\Lib\\site-packages', 'c:\\Users\\Administrator\\anaconda3\\Lib\\site-packages\\win32', 'c:\\Users\\Administrator\\anaconda3\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\Administrator\\anaconda3\\Lib\\site-packages\\Pythonwin', 'c:\\Users\\Administrator\\anaconda3\\Lib\\site-packages\\setuptools\\_vendor', '../..', 'f:\\py\\nemat\\minst\\test']


In [2]:
male_heights = np.random.normal(171, 6, 500)
female_heights = np.random.normal(158, 5, 500)

male_weights = np.random.normal(70, 10, 500)
female_weights = np.random.normal(57, 8, 500)

male_bfrs = np.random.normal(16, 2, 500)
female_bfrs = np.random.normal(22, 2, 500)

male_labels = [1] * 500
female_labels = [-1] * 500

train_set = np.array([np.concatenate((male_heights, female_heights)),
                      np.concatenate((male_weights, female_weights)),
                      np.concatenate((male_bfrs, female_bfrs)),
                      np.concatenate((male_labels, female_labels))]).T
np.random.shuffle(train_set)
train_set


array([[167.23538765,  60.73903323,  20.32274359,  -1.        ],
       [170.42638314,  57.44198182,  19.56637244,   1.        ],
       [147.4547613 ,  63.24510176,  23.62773432,  -1.        ],
       ...,
       [164.55271904,  53.98244422,  23.44890777,  -1.        ],
       [163.36799996,  70.22057214,  19.15884125,  -1.        ],
       [172.65341811,  66.76300295,  15.54057268,   1.        ]])

In [10]:
batch_length = len(train_set)
# 构造计算图：输入向量，是一个100x1矩阵，不需要初始化，不参与训练
x =variable_node.Variable(dim=(batch_length, 3), init=False, trainable=False)
x.value = train_set[:, 0:3]
# 类别标签，1男，-1女
label =variable_node.Variable(dim=(batch_length, 1), init=False, trainable=False)
label.value = train_set[:, -1]
# print(label.value)
# print(x.value)

# 权重向量，是一个1x3矩阵，需要初始化，参与训练
w =variable_node.Variable(dim=(3, 1), init=True, trainable=True)

# 阈值，是一个1x1矩阵，需要初始化，参与训练
b =variable_node.Variable(dim=(1, 1), init=True, trainable=True)
print(x.value,w.value)

[[167.23538765  60.73903323  20.32274359]
 [170.42638314  57.44198182  19.56637244]
 [147.4547613   63.24510176  23.62773432]
 ...
 [164.55271904  53.98244422  23.44890777]
 [163.36799996  70.22057214  19.15884125]
 [172.65341811  66.76300295  15.54057268]] [[ 0.0011607 ]
 [-0.00159247]
 [-0.00052664]]


In [11]:
# 构造计算图：xw是一个100x1矩阵，表示输入向量与权重向量的乘积，参与训练
xw=operate_node.MatMul(x, w)  
print(xw.value)
output = operate_node.Add(xw, b)
output.value

None


In [5]:

predict = activity_node.Step(output)

# 损失函数
#loss = ms.ops.loss.PerceptionLoss(ms.ops.MatMul(label, output))
output.value=output.value.T
loss=loss_node.LogLoss(operate_node.MatMul(label, output))
default_graph = graph.default_graph
loss.forward()
print("loss:", loss.value)

AttributeError: 'NoneType' object has no attribute 'T'

In [ ]:

        # 在loss节点上执行前向传播，计算损失值
        loss.forward()
        print("这是第",epoch+1,"轮向前传播的结果，以下是各节点的值：")
        for n in default_graph.nodes:
            print(n.node_name, ":", n.value)

        # 在w和b节点上执行反向传播，计算损失值对它们的雅可比矩阵
        w.backward(loss)
        b.backward(loss)

        w.set_value(w.value - learning_rate * w.jacobi)
        b.set_value(b.value - learning_rate * b.jacobi)
        print('现在是执行了反向传播后的各节点的值：')
        for n in default_graph.nodes:
            print(n.node_name, ":", n.value)
        # default_graph对象保存了所有节点，调用clear_jacobi方法清除所有节点的雅可比矩阵
        default_graph.clear_jacobi()

    # 每个epoch结束后评价模型的正确率
    pred = []

    # 遍历训练集，计算当前模型对每个样本的预测值
    for i in range(len(train_set)):
        
        features = np.mat(train_set[i, :-1]).T
        x.set_value(features)

        # 在模型的predict节点上执行前向传播
        predict.forward()
        pred.append(predict.value[0, 0])  # 模型的预测结果：1男，0女

    pred = np.array(pred) * 2 - 1  # 将1/0结果转化成1/-1结果，好与训练标签的约定一致

    # 判断预测结果与样本标签相同的数量与训练集总数量之比，即模型预测的正确率
    accuracy = (train_set[:, -1] == pred).astype(np.int32).sum() / len(train_set)

    # 打印当前epoch数和模型在训练集上的正确率
    print("epoch: {:d}, accuracy: {:.3f}".format(epoch + 1, accuracy))
    
    
    
